In [20]:
using BenchmarkTools

In [21]:
function rotate(matrix)
    return reverse(reverse(matrix, dims=1), dims=2)
end

rotate (generic function with 1 method)

In [22]:
function Convolution_2d_pad(input, kernel; bias=0., padding=false)
    input_rows, input_columns = size(input)
    kernel_height, kernel_width = size(kernel)

    if padding
        padded_input = zeros(Float32, input_rows + 2*kernel_height - 2, input_columns + 2*kernel_width - 2)
        padded_input[kernel_height:end-kernel_height+1, kernel_width:end-kernel_width+1] .= input
        input_rows, input_columns = size(padded_input)
        input = padded_input
    end

    output_rows = input_rows - kernel_height + 1
    output_columns = input_columns - kernel_width + 1
    output = zeros(Float32, output_rows, output_columns)


    for c in 1:output_columns
        for r in 1:output_rows
            patch = @view input[r:r+kernel_height-1, c:c+kernel_width-1]
            output[r, c] = sum(patch .* kernel) + bias
        end
    end
    return output
end

Convolution_2d_pad (generic function with 1 method)

In [23]:
function Convolution_2d(input, kernel; bias=0.)
    input_rows, input_columns = size(input)
    kernel_height, kernel_width = size(kernel)

    output_rows = input_rows - kernel_height + 1
    output_columns = input_columns - kernel_width + 1
    output = zeros(Float32, output_rows, output_columns)


    for c in 1:output_columns
        for r in 1:output_rows
            patch = @view input[r:r+kernel_height-1, c:c+kernel_width-1]
            output[r, c] = sum(patch .* kernel) + bias
        end
    end
    return output
end

Convolution_2d (generic function with 1 method)

In [24]:
function Convolution_2d!(ret, input, kernel; bias=0., padding=false)
    input_rows, input_columns = size(input)
    kernel_height, kernel_width = size(kernel)

    if padding
        padded_input = zeros(Float32, input_rows + 2*kernel_height - 2, input_columns + 2*kernel_width - 2)
        padded_input[kernel_height:end-kernel_height+1, kernel_width:end-kernel_width+1] .= input
        input_rows, input_columns = size(padded_input)
        input = padded_input
    end

    output_rows = input_rows - kernel_height + 1
    output_columns = input_columns - kernel_width + 1
    sumret = zeros(size(kernel))
    for c in 1:output_columns
        for r in 1:output_rows
            patch = @view input[r:r+kernel_height-1, c:c+kernel_width-1]
            sumret .= patch .* kernel
            ret[r, c] = sum(sumret) + bias
            sumret .= 0.0
        end
    end
end

Convolution_2d! (generic function with 1 method)

In [25]:
function Conv_backward_v1( input, weights, bias, gradient)
    input_height, input_width, input_channels = size(input)
    output_height, output_width, output_channels = size(gradient)
    kernel_height, kernel_width, _, _ = size(weights)
    
    
    grad_input = zeros(Float32, size(input))
    for k in 1:input_channels
        for c in 1:output_channels
            grad_input[:, :, k] += Convolution_2d_pad(weights[:, :, k, c], gradient[:, :, c]; padding=true)
        end
    end

    grad_weights = zeros(Float32, size(weights))
    for k in 1:input_channels
        for c in 1:output_channels
            grad_weights[:, :, k, c] += Convolution_2d(input[:, :, k], gradient[:, :, c])
        end
    end

    grad_bias = reshape(sum(gradient, dims=(1,2,4)), :)
    
    return grad_input, grad_weights, grad_bias
end

Conv_backward_v1 (generic function with 1 method)

In [26]:
function Conv_backward_v2(input, weights, bias, gradient)
    input_height, input_width, input_channels = size(input)
    output_height, output_width, output_channels = size(gradient)
    kernel_height, kernel_width, _, _ = size(weights)
    
    
    grad_input = zeros(Float32, size(input))
    grad_weights = zeros(Float32, size(weights))
    
    tmp_weights = zeros(Float32, kernel_height, kernel_width)
    tmp_input = zeros(Float32, input_height, input_width)
    tmp_gradient = zeros(Float32, output_height, output_width)
    
    for k in 1:input_channels
        for c in 1:output_channels
            tmp_weights .= @views weights[:, :, k, c]
            tmp_gradient .= @views gradient[:, :, c]
            grad_input[:, :, k] .+= Convolution_2d_pad(tmp_weights, rotate(tmp_gradient); padding=true)
        end
    end

    
    for k in 1:input_channels
        for c in 1:output_channels
            tmp_input .= @views input[:, :, k]
            tmp_gradient .= @views gradient[:, :, c]
            grad_weights[:, :, k, c] += Convolution_2d(tmp_input, tmp_gradient)
        end
    end

    grad_bias = reshape(sum(gradient, dims=(1,2,4)), :)
    
    return grad_input, grad_weights, grad_bias
end

Conv_backward_v2 (generic function with 1 method)

In [27]:
function Conv_backward_v3( input, weights, bias, gradient)
    input_height, input_width, input_channels = size(input)
    output_height, output_width, output_channels = size(gradient)
    kernel_height, kernel_width, _, _ = size(weights)
    
    grad_input = zeros(Float32, size(input))
    grad_weights = zeros(Float32, size(weights))
    
    tmp_weights = zeros(Float32, kernel_height, kernel_width)
    tmp_input = zeros(Float32, input_height, input_width)
    tmp_gradient = zeros(Float32, output_height, output_width)
    
    for k in 1:input_channels
        for c in 1:output_channels
            tmp_weights .= @views weights[:, :, k, c]
            tmp_gradient .= @views gradient[:, :, c]
            for i = 1:output_height
                for j = 1:output_width
                    grad_input[i:i+kernel_height-1, j:j+kernel_width-1, k] .+= (tmp_weights .* tmp_gradient[i,j]);
                end
            end
        end
    end
    
    for k in 1:input_channels
        for c in 1:output_channels
            tmp_input .= @views input[:, :, k]
            tmp_gradient .= @views gradient[:, :, c]
            grad_weights[:, :, k, c] += Convolution_2d(tmp_input, tmp_gradient)
        end
    end

    grad_bias = reshape(sum(gradient, dims=(1,2,4)), :)
    
    return grad_input, grad_weights, grad_bias
end

Conv_backward_v3 (generic function with 1 method)

In [28]:
function Conv_backward_v4(input, weights, bias, gradient)
    input_height, input_width, input_channels = size(input)
    output_height, output_width, output_channels = size(gradient)
    kernel_height, kernel_width, _, _ = size(weights)
    
    grad_input = zeros(Float32, size(input))
    grad_weights = zeros(Float32, size(weights))
    
    tmp_weights = zeros(Float32, kernel_height, kernel_width)
    tmp_input = zeros(Float32, input_height, input_width)
    tmp_gradient = zeros(Float32, output_height, output_width)
    
    for k in 1:input_channels
        for c in 1:output_channels
            tmp_weights .= @views weights[:, :, k, c]
            tmp_gradient .= @views gradient[:, :, c]
            for i = 1:output_height
                for j = 1:output_width
                    grad_input[i:i+kernel_height-1, j:j+kernel_width-1, k] .+= (tmp_weights .* tmp_gradient[i,j]);
                end
            end
        end
    end
    
    for k in 1:input_channels
        for c in 1:output_channels
            tmp_input .= @views input[:, :, k]
            tmp_gradient .= @views gradient[:, :, c]
            Convolution_2d!(tmp_weights, tmp_input, tmp_gradient)
            grad_weights[:, :, k, c] .+= tmp_weights
            tmp_weights .= 0
        end
    end

    grad_bias = reshape(sum(gradient, dims=(1,2,4)), :)
    
    return grad_input, grad_weights, grad_bias
end

Conv_backward_v4 (generic function with 1 method)

In [29]:
function Conv_backward_v5(input, weights, bias, gradient)
  input_height, input_width, input_channels = size(input)
  output_height, output_width, output_channels = size(gradient)
  kernel_height, kernel_width, _, _ = size(weights)
  
  grad_input = zeros(Float32, size(input))
  grad_weights = zeros(Float32, size(weights))
  
  tmp_weights = zeros(Float32, kernel_height, kernel_width)
  tmp_input = zeros(Float32, input_height, input_width)
  tmp_gradient = zeros(Float32, output_height, output_width)
  tmp_weight_gradient = zeros(Float32, kernel_height, kernel_width)
  for k in 1:input_channels
      for c in 1:output_channels
          for i = 1:output_height
              for j = 1:output_width
                  @views grad_input[i:i+kernel_height-1, j:j+kernel_width-1, k] .+= (weights[:, :, k, c] .* gradient[i, j, c]);
              end
          end
      end
  end
  
  for k in 1:input_channels
      for c in 1:output_channels
          tmp_weights .= 0
          tmp_input .= @views input[:, :, k]
          tmp_gradient .= @views gradient[:, :, c]
          Convolution_2d!(tmp_weights, tmp_input, tmp_gradient)
          grad_weights[:, :, k, c] .+= tmp_weights
      end
  end

  grad_bias = reshape(sum(gradient, dims=(1,2,4)), :)
  
  return grad_input, grad_weights, grad_bias
end

Conv_backward_v5 (generic function with 1 method)

In [30]:
function Conv_backward_v6(input, weights, bias, gradient)
  input_height, input_width, input_channels = size(input)
  output_height, output_width, output_channels = size(gradient)
  kernel_height, kernel_width, _, _ = size(weights)
  
  grad_input = zeros(Float32, size(input))
  grad_weights = zeros(Float32, size(weights))
  
  tmp_weights = zeros(Float32, kernel_height, kernel_width)
  tmp_input = zeros(Float32, input_height, input_width)
  tmp_gradient = zeros(Float32, output_height, output_width)
  tmp_weight_gradient = zeros(Float32, kernel_height, kernel_width)
  for k in 1:input_channels
      for c in 1:output_channels
          for i = 1:output_height
              for j = 1:output_width
                  @views grad_input[i:i+kernel_height-1, j:j+kernel_width-1, k] .+= (weights[:, :, k, c] .* gradient[i, j, c]);
              end
          end
      end
  end
  
  for k in 1:input_channels
      for c in 1:output_channels
          tmp_weights .= 0
          tmp_input .= @views input[:, :, k]
          tmp_gradient .= @views gradient[:, :, c]
          Convolution_2d!(tmp_weights, tmp_input, tmp_gradient)
          @views grad_weights[:, :, k, c] .+= tmp_weights
      end
  end

  grad_bias = reshape(sum(gradient, dims=(1,2,4)), :)
  
  return grad_input, grad_weights, grad_bias
end

Conv_backward_v6 (generic function with 1 method)

In [31]:
input = rand(28, 28, 1);
weights = rand(3, 3, 1, 6);
bias = rand(Float32, 6);
gradient = rand(26, 26, 6);

In [32]:
@benchmark Conv_backward_v1(input, weights, bias, gradient)

BenchmarkTools.Trial: 1139 samples with 1 evaluation.
 Range (min … max):  2.858 ms … 15.653 ms  ┊ GC (min … max):  0.00% … 35.85%
 Time  (median):     4.231 ms              ┊ GC (median):    24.27%
 Time  (mean ± σ):   4.382 ms ±  1.508 ms  ┊ GC (mean ± σ):  17.91% ± 13.65%

  ▇█▅    ▃▄▃▃▅▁                                               
  ███▆▄▄▆███████▄▅▄▄▄▃▃▃▃▃▂▃▃▃▂▁▂▂▂▂▂▁▂▂▂▁▁▂▂▁▂▂▁▂▁▁▁▂▁▂▂▁▂▂ ▃
  2.86 ms        Histogram: frequency by time        11.9 ms <

 Memory estimate: 25.49 MiB, allocs estimate: 4842.

In [33]:
@benchmark Conv_backward_v2(input, weights, bias, gradient)

BenchmarkTools.Trial: 1295 samples with 1 evaluation.
 Range (min … max):  2.747 ms … 12.405 ms  ┊ GC (min … max):  0.00% … 46.79%
 Time  (median):     3.205 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   3.851 ms ±  1.308 ms  ┊ GC (mean ± σ):  16.42% ± 18.25%

  ▅█▅▄▄▂▃▃▂        ▄▄▃▃▁▁▁▁▁ ▁ ▁                              
  ██████████▇▅▇▅▅▅▇██████████████▆█▆▇▅▅▁▁▅▅▅▅▆▄▅▅▅▁▁▅▁▅▁▄▄▁▅ █
  2.75 ms      Histogram: log(frequency) by time     8.75 ms <

 Memory estimate: 13.22 MiB, allocs estimate: 4827.

In [34]:
@benchmark Conv_backward_v3(input, weights, bias, gradient)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  306.000 μs …   3.194 ms  ┊ GC (min … max): 0.00% … 87.94%
 Time  (median):     316.500 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   346.045 μs ± 140.326 μs  ┊ GC (mean ± σ):  3.73% ±  8.03%

  █▆▄▃▂▁▁                                                       ▁
  ████████████▆▇█▆▇▆▆▇▇▇▆▆▆▅▅▅▆▄▄▁▁▄▁▁▃▄▁▃▃▃▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▁▁▅▇ █
  306 μs        Histogram: log(frequency) by time       1.22 ms <

 Memory estimate: 544.08 KiB, allocs estimate: 4149.

In [35]:
@benchmark Conv_backward_v4(input, weights, bias, gradient)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  307.000 μs …   2.263 ms  ┊ GC (min … max): 0.00% … 61.65%
 Time  (median):     316.400 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   364.784 μs ± 124.790 μs  ┊ GC (mean ± σ):  1.88% ±  6.20%

  █▅▄▄▃▃▂▂▂▁▁▁                                                  ▁
  █████████████████▇▇▇▇▇▆▇▇▆▇▇▇▇▇▇▇▇███▇▆▇▇▅▅▆▅▅▄▅▅▃▃▄▄▅▄▃▃▃▃▄▅ █
  307 μs        Histogram: log(frequency) by time        893 μs <

 Memory estimate: 423.42 KiB, allocs estimate: 4083.

In [36]:
@benchmark Conv_backward_v5(input, weights, bias, gradient)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  126.900 μs …  6.004 ms  ┊ GC (min … max): 0.00% … 97.02%
 Time  (median):     130.400 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   137.943 μs ± 77.651 μs  ┊ GC (mean ± σ):  1.35% ±  2.72%

  █▇▇▆▅▅▄▃▃▃▃▂▁▁▁                                              ▂
  ███████████████████▇▇▇█▇▇▆▆▆▆▆▆▆▅▅▅▅▄▅▆▆▅▇▇▇▆▅▄▄▅▄▆▅▆▅▅▅▅▄▆▆ █
  127 μs        Histogram: log(frequency) by time       224 μs <

 Memory estimate: 43.27 KiB, allocs estimate: 28.

In [37]:
@benchmark Conv_backward_v6(input, weights, bias, gradient)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  127.600 μs …  2.054 ms  ┊ GC (min … max): 0.00% … 90.48%
 Time  (median):     130.800 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   136.289 μs ± 49.162 μs  ┊ GC (mean ± σ):  1.00% ±  2.71%

  █▆█▆▅▅▅▄▃▃▃▃▃▂▂▂▂▂▁▁                                         ▂
  ██████████████████████▇▇▇█▆▇█▇▇▇▇▆▇▇▇▆▆▅▅▆▅▃▄▅▅▁▅▅▆▄▅▅▅▄▅▄▅▄ █
  128 μs        Histogram: log(frequency) by time       202 μs <

 Memory estimate: 42.70 KiB, allocs estimate: 22.

In [38]:
Conv_backward_v4(input, weights, bias, gradient)

(Float32[2.020429 4.06357 … 2.2930114 0.8286258; 3.9139562 7.285532 … 6.109718 2.6047778; … ; 3.5319715 8.9466915 … 7.8518877 2.8365035; 1.3006811 4.4725003 … 3.6390018 1.2054188;;;], Float32[164.40111 165.36755 161.70302; 162.04048 168.52216 166.49551; 163.02325 162.58156 162.8977;;;; 171.80312 180.19992 178.05803; 176.86284 175.27556 173.06384; 174.33675 172.52469 171.81917;;;; 172.902 175.74791 173.22466; 174.04797 175.17746 172.57898; 173.50313 171.99965 175.20943;;;; 165.05063 169.43669 166.0624; 162.2465 164.50232 168.04941; 170.24167 164.53468 161.58568;;;; 168.50557 172.65413 170.27284; 170.57567 169.61488 171.79158; 168.28915 172.25029 169.6158;;;; 170.5347 170.29628 171.6186; 174.53009 170.05519 171.70311; 168.23741 168.62231 173.56142], [331.72709829916676, 350.0516172318312, 344.8505915174053, 334.3280651932331, 337.20880731325656, 342.1236136561699])

In [39]:
Conv_backward_v5(input, weights, bias, gradient)

(Float32[2.0204287 4.0635705 … 2.2930114 0.8286258; 3.9139562 7.2855315 … 6.1097183 2.604778; … ; 3.5319715 8.9466915 … 7.8518887 2.8365033; 1.300681 4.4725003 … 3.6390018 1.205419;;;], Float32[164.40111 165.36755 161.70302; 162.04048 168.52216 166.49551; 163.02325 162.58156 162.8977;;;; 171.80312 180.19992 178.05803; 176.86284 175.27556 173.06384; 174.33675 172.52469 171.81917;;;; 172.902 175.74791 173.22466; 174.04797 175.17746 172.57898; 173.50313 171.99965 175.20943;;;; 165.05063 169.43669 166.0624; 162.2465 164.50232 168.04941; 170.24167 164.53468 161.58568;;;; 168.50557 172.65413 170.27284; 170.57567 169.61488 171.79158; 168.28915 172.25029 169.6158;;;; 170.5347 170.29628 171.6186; 174.53009 170.05519 171.70311; 168.23741 168.62231 173.56142], [331.72709829916676, 350.0516172318312, 344.8505915174053, 334.3280651932331, 337.20880731325656, 342.1236136561699])